# SNS Sentiment Analysis

Analyze social media texts and measure potential inflammatory / offensive language.

# Data Pre-processing

From the selected datasets, extract the text and labels from all of them, then combine into one large CSV dataset.

(Install NLTK data if not already installed).

In [1]:
import nltk, os

# Run this if you are locally accessing the NLTK data
nltk.data.path.append('./nltk_data/')
if not os.path.exists('./nltk_data'):
    nltk.download('punkt', download_dir='./nltk_data/')
    nltk.download('stopwords', download_dir='./nltk_data/')
    nltk.download('words', download_dir='./nltk_data/')
    nltk.download('brown', download_dir='./nltk_data/')

In [2]:
hate_speech_dataset_path = "./datasets/hate_speech_detect/HateSpeechDatasetBalanced.csv"
malignant_dataset_path = "./datasets/malignant/train.csv"

In [7]:
import data_util as du

In [ ]:
hs_tuples = du.generate_tuples_from_file(hate_speech_dataset_path)

In [ ]:
m_tuples = du.generate_tuples_from_df(processed_m_train_df)

In [ ]:
# Combine and save the data to a CSV
processed_data_save = pd.DataFrame({
    "text": hs_tuples[0] + m_tuples[0],
    "label": hs_tuples[1] + m_tuples[1]
})

processed_data_save.to_csv("./datasets/processed/all_data.csv", index=False)

# Load data and split